In [ ]:
git clone https://github.com/destabilizer/prozhito-tools

In [1]:
#импортируем тулзу и заwrap'им прожито
import dump
dw = dump.Wrapper(csvpath='/Users/kirillkonca1/Downloads/prozhito-dump-1/')

Теперь нужно выбрать запись. Раз запись должна быть достаточно далекой, можно посмотреть записи с августа 1800 года до конца 1805-го. Кажется, 200 лет достаточно большая разница, к тому же в таком промежутке достаточно много записей, так что будет из чего выбрать.

In [2]:
notes = dw.notes[(1800, 8, 1) : (1805, 12, 31)]

In [3]:
note = notes[10].text
note

'Берлин    \n[...] Сюда въехали в час пополудни, остановились в «Золотом Солнце», под названием «Российской Постоялый двор». На самой лучшей улице, где гульбище, по-немецки Linden Strasse, т. е. Липовая улица. Это то же, что наш Невский проспект, но короче, домы не так хороши, хотя некоторые выше петербургских [...] Мы своими зелеными шапками удивили весь Берлин и сапогами также. Все, встречавшие нас, шептали: «Вот русские шапки и сапоги», а смотрели на нас, как на чучел [...] Ходили по городу, имели счастие быть заманиваемы берлинскими нимфами и очень приманчивыми. Но мы с И[ваном] П[етровичем Ленцем], как новые Улиссы, отвращали глаза и пробегали, подобно как царь Итакский, затыкая уши, чтоб не слышать очаровательного пения сирен.    \nГород прекрасен, — вот первый по выезде из любезного Петербурга, который мне понравился, но прочие все дрянь. Как в деревнях по Пруссии, так и в городах по большей части строения мазанковые. Реки, каналы — все есть, но не Нева, Фонтанка, Екатерининский

Я решил выбрать данный отрывок из-за нескольких причин. Во-первых, он достаточно целостный семантически и в принципе самодостаточный. Во-вторых, заимствования. От текстов этих времен закономерно ожидать немецких и французских заимствований (возможно еще латинских и древнегреческих, тут уже важна специфика текста). Как можно видеть, в тексте есть прямая вставка на немецком: _Linden Strasse_. В-третьих, тут достаточно много локаций и есть имена, поэтому будет что размечать. И еще: язык этого текста хоть и понятен нам, тем не менее содержит несколько необычных деталей. Например, необычное использование падежей в некоторых случаях (_домы_), я не думаю, что это сильно повлияет на NER-разметку, но все возможно.

Ещё стоит упомянуть само оформление записей в «Прожито»: часто информация там сопровождается ссылками (<com id=...>), квадратными скобками (как в тексте выше) и иногда пометами *типа таких*. Естественно, это не может не влиять на то, как будет работать NER-тэггер, поэтому нужно будет преобразовывать текст, чтобы наиболее точно извлекать из него информацию. Для NER-тэггинга я вручную убрал из текста квадратные скобки в имени Ивана Петровича Ленца.

In [14]:
text = 'Берлин. Сюда въехали в час пополудни, остановились в «Золотом Солнце», под названием «Российской Постоялый двор». \
На самой лучшей улице, где гульбище, по-немецки Linden Strasse, т. е. Липовая улица. Это то же, что наш \
Невский проспект, но короче, домы не так хороши, хотя некоторые выше петербургских [...] Мы своими зелеными шапками \
удивили весь Берлин и сапогами также. Все, встречавшие нас, шептали: «Вот русские шапки и сапоги», а смотрели на нас, как на чучел [...] \
Ходили по городу, имели счастие \
быть заманиваемы берлинскими нимфами и очень приманчивыми. Но мы с Иваном Петровичем Ленцем, как новые \
Улиссы, отвращали глаза и пробегали, подобно как царь Итакский, затыкая уши, чтоб не слышать очаровательного \
пения сирен.    \nГород прекрасен, — вот первый по выезде из любезного Петербурга, который мне понравился, \
но прочие все дрянь. Как в деревнях по Пруссии, так и в городах по большей части строения мазанковые. \
Реки, каналы — все есть, но не Нева, Фонтанка, Екатерининский канал и Мойка [...]    \nВ рассуждении театра \
я опоздал: время опер уже прошло, следовательно и танцоров здешних не увижу, ибо особых балетов, кроме \
аналогических, здесь уже давно нет [...]'


In [15]:
#В качестве NER-тэггера я выбрал natasha.
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [24]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

Я думаю, что в разметке этого куска текста ошибок не будет. Разве что «Золотое Солнце» и «Российской Постоялый двор» не будут как-то размечены (нет явных указателей на то, чем являются эти сущности, к тому же natasha тренировалась на текстах современных новостей, а в наше время вряд ли эти имена употребляются). Естественно, что _Linden Strasse_ тоже никак не будет размечена. 

In [25]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Берлин. Сюда въехали в час пополудни, остановились в «Золотом Солнце»,
LOC───                                                                
 под названием «Российской Постоялый двор». На самой лучшей улице, где
 гульбище, по-немецки Linden Strasse, т. е. Липовая улица. Это то же, 
                                            LOC──────────             
что наш Невский проспект, но короче, домы не так хороши, хотя 
        LOC─────────────                                      
некоторые выше петербургских [...] Мы своими зелеными шапками удивили 
весь Берлин и сапогами также. Все, встречавшие нас, шептали: «Вот 
     LOC───                                                       
русские шапки и сапоги», а смотрели на нас, как на чучел [...] Ходили 
по городу, имели счастие быть заманиваемы берлинскими нимфами и очень 
приманчивыми. Но мы с Иваном Петровичем Ленцем, как новые Улиссы, 
                      PER─────────────────────                    
отвращали глаза и пробегали, подобно к

Как можно видеть из этой разметки, Берлин, Петербург, Липовую улицу, Невский проспект, Ивана Петровича Ленца, Пруссию, Екатерининский канал и Мойку natasha разметила верно. Улиссы как PER тэггер не разметил, а Неву и Фонтанку вообще пометил как организации. В чем может быть причина этого? Мне кажется, прежде всего, в том, что NER-тэггеры обучаются на современных текстах. Помимо того, что имя достаточно редкое и обычно употребляются в контексте названия романа, но еще и ситуация при котором имя ставится во множественное число не такая обычная для современных текстов, поэтому велика вероятность, что NER-тэггеры будут пропускать такие вещи. По этой же причине Фонтанка и Нева размечены как ORG. Так, например, сейчас Фонтанка может употребляться как название газеты, а название Нева наверняка носит какая-нибудь известная кампания (а то и не одна).

В целом, конечно, natasha более-менее хорошо определяет NER-тэги, как никак язык хоть и отличается, достаточно похож. Тем не менее, чтобы улучшить результат работы наверняка придется размечать вручную корпус старых текстов и обучать на нем тэггер, чтобы таких ошибок как в этом случае не возникало. К тому же надо будет научиться размечать иностранные названия: отличать имена от мест/организаций и цитат/заимствований, так как иноязычные вкрапления не редкость в текстах тех времен (да и этих тоже). Также надо будет посмотреть на разные необычные для наших времен употребления имен (как, например, когда одно только имя употреблено в мн.ч: Улиссы, Иваны и т.д.). Но это всего лишь то, что лежит на поверхности, я уверен, что придется столкнуться со множеством мелких деталей при дальнейшей работе.